In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
# Importando datos 
dataset_train = pd.read_csv('./deeplearning-az/datasets/Part 3 - Recurrent Neural Networks (RNN)/Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

# Escalando datos
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Restrucurando datos a N periodos
periods = 120
X_train = []
y_train = []
for i in range(periods, 1258):
    X_train.append(training_set_scaled[i-periods:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Restructurando datos a 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [21]:
# Creando arquitectura RNR

def create_model(neurons=50, optimizer='adam'):
    regressor = Sequential()
    regressor.add(LSTM(units = neurons, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = neurons*2, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = neurons*2, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = neurons*4, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = neurons*4))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units=50, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))

    # Compilando
    regressor.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics=['mean_absolute_percentage_error'])

    return regressor

regressor = create_model(500, 'adam')
regressor.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 120, 500)          1004000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 120, 500)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 120, 1000)         6004000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 120, 1000)         0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 120, 1000)         8004000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 120, 1000)         0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 120, 2000)        

In [22]:
# Definir callback para reducir learning rate

import os
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR) :
    os.mkdir(MODEL_DIR)
modelpath = './model/rnr-google-{epoch:02d}-{loss:.4f}.hdf5'

checkPointer = ModelCheckpoint(filepath=modelpath, monitor='loss', verbose=1, save_best_only=True)
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

callbacks=[earlystop, learning_rate_reduction, checkPointer]

In [ ]:
# Entrenar
regressor.fit(X_train, 
                y_train,
                epochs=50,
                batch_size = 32,
                callbacks=callbacks,
                use_multiprocessing=True,
                workers=16)


Epoch 1/50
1138/1138 [==============================] - 83s 73ms/step - loss: 8.1062 - mean_absolute_percentage_error: 355.4794

Epoch 00001: loss improved from inf to 8.10619, saving model to ./model/rnr-google-01-8.1062.hdf5


/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mean_absolute_percentage_error
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/50
1138/1138 [==============================] - 81s 71ms/step - loss: 0.0730 - mean_absolute_percentage_error: 143.4947

Epoch 00002: loss improved from 8.10619 to 0.07295, saving model to ./model/rnr-google-02-0.0730.hdf5
Epoch 3/50
1138/1138 [==============================] - 81s 71ms/step - loss: 0.0240 - mean_absolute_percentage_error: 75.5375

Epoch 00003: loss improved from 0.07295 to 0.02396, saving model to ./model/rnr-google-03-0.0240.hdf5
Epoch 4/50
1138/1138 [==============================] - 81s 71ms/step - loss: 0.0111 - mean_absolute_percentage_error: 35.3677

Epoch 00004: loss improved from 0.02396 to 0.01109, saving model to ./model/rnr-google-04-0.0111.hdf5
Epoch 5/50
1138/1138 [==============================] - 81s 71ms/step - loss: 0.0117 - mean_absolute_percentage_error: 45.7715

Epoch 00005: loss did not improve from 0.01109
Epoch 6/50
1138/1138 [==============================] - 81s 71ms/step - loss: 0.0095 - mean_absolute_percentage_error: 38.2212

Epoch 

In [5]:
# Entrenar con GridSearchCV
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor 


model = KerasRegressor (build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = dict(
#    batch_size=[32, 64], 
#    epochs=[50],
    neurons=[300, 1000],
    optimizer=['adam', 'adadelta']
)

grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid,  
                    cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: 

In [ ]:
# Validando

dataset_test = pd.read_csv('./deeplearning-az/datasets/Part 3 - Recurrent Neural Networks (RNN)/Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - periods:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(periods, periods+20):
    X_test.append(inputs[i-periods:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visaulizando

plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

In [ ]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
print('Score: %.2f RMSE' % (rmse))

from sklearn.metrics import r2_score
r2 = r2_score(real_stock_price, predicted_stock_price)
print('Score: %.2f R2' % (r2))

In [ ]:
real_stock_price[15]

In [ ]:
predicted_stock_price[15]

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2515935398727473935
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4175418742581851552
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16310127797302392875
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7510514074
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10398471313992723394
physical_device_desc: "device: 0, name: Tesla M60, pci bus id: 0000:00:1e.0, compute capability: 5.2"
]


In [11]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

print(_get_available_gpus())

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf
['/device:GPU:0']


In [16]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1
